In [1]:
# import json data to pandas
import pandas as pd
import json
import os
import numpy as np
import asyncio
import aiohttp

# rerank

In [34]:
valid_rank_data = pd.read_json('data_and_code/zqy_data/faiss_results.jsonl', lines=True)
valid_rank_data

,question,answer,document_id
0,when did the british first land in north america,NaN,"[11484, 2724, 10057, 4117, 81, 9798, 570, 1013..."
1,when did the 1st world war officially end,NaN,"[6909, 7427, 10722, 11362, 7899, 1896, 646, 97..."
2,who's the girl that plays the new wonder woman,NaN,"[5970, 8669, 8552, 10380, 10786, 11164, 10484,..."
3,who is the director of the cia today,NaN,"[11721, 4733, 3595, 8855, 11334, 4857, 9619, 8..."
4,who plays ben in the new fantastic four,NaN,"[8635, 9192, 9792, 6860, 4805, 9889, 9625, 555..."
...,...,...,...
995,who did morgan freeman play in driving miss daisy,NaN,"[2987, 7099, 7797, 5287, 4261, 785, 5692, 7364..."
996,who played mr kidd in diamonds are forever,NaN,"[7690, 6834, 628, 10418, 10227, 2057, 3517, 16..."
997,when does nathan find out about chuck and blair,NaN,"[9847, 3759, 2452, 6304, 4191, 6249, 4622, 952..."
998,when did the first push to start car come out,NaN,"[2352, 271, 4740, 5964, 6329, 11276, 10476, 96..."


In [35]:
document_data = pd.read_json('data_and_code/ytx_data/processed_markdown.jsonl', lines=True)
document_plain = pd.read_json('data_and_code/ytx_data/processed_plain.jsonl', lines=True)
document_data

,doc_id,text
0,0,Email marketing - Wikipedia # Email marketing ...
1,1,The Mother ( How I Met Your Mother ) - wikiped...
2,2,Human fertilization - wikipedia # Human fertil...
3,3,List of National Football League career quarte...
4,4,Roanoke Colony - wikipedia # Roanoke Colony ``...
...,...,...
12133,12133,Jeepers Creepers ( 2001 film ) - wikipedia # J...
12134,12134,Death Wish ( 2018 film ) - wikipedia # Death W...
12135,12135,X-Men : Apocalypse - wikipedia # X-Men : Apoca...
12136,12136,Gulliver 's Travels - wikipedia # Gulliver 's ...


In [36]:
# get max number of tokens in one document
max(document_data['text'].apply(lambda x: len(x.split(' ')))), \
    max(document_plain['text'].apply(lambda x: len(x.split(' '))))

(25123, 38117)

In [37]:
def idx2content(idx, document_data=document_data):
    """
    get document content by index
    """
    return document_data['text'][idx]

In [38]:
# reset column 'answer' of valid_rank_data to None
valid_rank_data['answer'] = None

In [39]:
# rerank
from QA_System.backbone.rerank import ReRanker


def rerank_func(valid_rank_data, reranker: ReRanker):
    input_top_n = 50
    docid_origin_list = valid_rank_data['document_id'].apply(lambda x: x[:input_top_n])
    query_list = valid_rank_data['question']
    doc_content_origin_list = docid_origin_list.apply(lambda x: [idx2content(int(i)) for i in x])
    # transfer to list
    doc_content_origin_list = doc_content_origin_list.tolist()
    query_list = query_list.tolist()
    docid_origin_list = docid_origin_list.tolist()
    # combine query and document, format: [query, list of documents content]
    query_document_list = []
    for i in range(len(query_list)):
        query_document_list.append(
            [
                query_list[i],
                doc_content_origin_list[i]
            ]
)
    # rerank
    responses = reranker.async_send_requests_simple(
        query_document_list, use_progress_bar=False, concurrency=5, top_n = 5, 
        model = 'Pro/BAAI/bge-reranker-v2-m3',
    )
    _, indices = reranker.extract_json(responses)
    # indices is the relative index of the document in the original list
    # transform to the original document id
    # docid_origin_list = list[list[int]]
    # indices = list[list[int]]
    for i in range(len(indices)):
        for j in range(len(indices[i])):
            indices[i][j] = docid_origin_list[i][indices[i][j]]
    return indices

In [40]:
import nest_asyncio
import time
nest_asyncio.apply()

api_key = "sk-tiqxioukbterlpaezfbeezpfkcpqbtaqtknvfjrrkscgtwmg"
reranker = ReRanker(api_token=api_key)
all_indices = []
# process 100 samples each time
batch_size = 30
for i in range(0, len(valid_rank_data), batch_size):
    print(f"Processing {i} to {i + batch_size}")
    indices = rerank_func(valid_rank_data[i:i + batch_size], reranker)
    all_indices.extend(indices)
    time.sleep(40)
# indices = rerank_func(valid_rank_data, reranker)

Processing 0 to 30
Processing 30 to 60
Processing 60 to 90
Processing 90 to 120
Processing 120 to 150
Processing 150 to 180
Processing 180 to 210
Processing 210 to 240
Processing 240 to 270
Processing 270 to 300
Processing 300 to 330
Processing 330 to 360
Processing 360 to 390
Processing 390 to 420
Processing 420 to 450
Processing 450 to 480
Processing 480 to 510
Processing 510 to 540
Processing 540 to 570
Processing 570 to 600
Processing 600 to 630
Processing 630 to 660
Processing 660 to 690
Processing 690 to 720
Processing 720 to 750
Processing 750 to 780
Processing 780 to 810
Processing 810 to 840
Processing 840 to 870
Processing 870 to 900
Processing 900 to 930
Processing 930 to 960
Processing 960 to 990
Processing 990 to 1020


In [ ]:
# # reformat str to int in all_indices
# all_indices = [[int(j) for j in i] for i in all_indices]
# all_indices

In [43]:
# replace the attribute of 'document_id' in valid_rank_data
# choose the first 5 documents_id
valid_rank_data['document_id'] = all_indices
valid_rank_data

,question,answer,document_id
0,when did the british first land in north america,None,"[11484, 2724, 12022, 570, 10133]"
1,when did the 1st world war officially end,None,"[10722, 9731, 7508, 646, 7880]"
2,who's the girl that plays the new wonder woman,None,"[7556, 10786, 5970, 10380, 8669]"
3,who is the director of the cia today,None,"[11721, 9619, 4715, 12126, 4733]"
4,who plays ben in the new fantastic four,None,"[9192, 8635, 5557, 10620, 9792]"
...,...,...,...
995,who did morgan freeman play in driving miss daisy,None,"[7099, 2987, 7797, 785, 7590]"
996,who played mr kidd in diamonds are forever,None,"[7690, 10227, 11126, 5356, 1051]"
997,when does nathan find out about chuck and blair,None,"[2452, 3759, 4191, 6249, 10832]"
998,when did the first push to start car come out,None,"[8331, 3525, 6907, 3617, 3279]"


In [44]:
all_indices_np = np.array(all_indices)
all_indices_np.shape

(1000, 5)

In [45]:
# save valid_rank_data
valid_rank_data.to_json('data_and_code/zqy_data/faiss_results_rerank2.jsonl', orient='records', lines=True)

# llm

In [6]:
from QA_System.backbone.llm import QwenChatClient
import nest_asyncio


In [49]:
valid_rank_data = pd.read_json('data_and_code/zqy_data/faiss_results_rerank.jsonl', lines=True)
valid_rank_data


,question,answer,document_id
0,when did the british first land in north america,NaN,"[11484, 2724, 12022, 570, 10133]"
1,when did the 1st world war officially end,NaN,"[10722, 9731, 7508, 646, 7880]"
2,who's the girl that plays the new wonder woman,NaN,"[7556, 10786, 5970, 10380, 8669]"
3,who is the director of the cia today,NaN,"[11721, 9619, 4715, 12126, 4733]"
4,who plays ben in the new fantastic four,NaN,"[9192, 8635, 5557, 10620, 9792]"
...,...,...,...
995,who did morgan freeman play in driving miss daisy,NaN,"[7099, 2987, 7797, 785, 7590]"
996,who played mr kidd in diamonds are forever,NaN,"[7690, 10227, 11126, 5356, 1051]"
997,when does nathan find out about chuck and blair,NaN,"[2452, 3759, 4191, 6249, 10832]"
998,when did the first push to start car come out,NaN,"[3525, 8331, 6907, 3617, 3279]"


In [50]:
def get_answer(client: QwenChatClient, valid_rank_data=valid_rank_data, idx:int = 0):
    """
    get answer from QwenChatClient
    """
    # get the first 5 document id
    docid_list = valid_rank_data['document_id']
    # get the question
    query_list = valid_rank_data['question']
    doc_content_list = docid_list.apply(lambda x: idx2content(int(x[idx])))
    # transfer to list
    doc_content_list = doc_content_list.tolist()
    query_list = query_list.tolist()
    # combine query and document, format: [query, documents content]
    query_context_list = []
    for i in range(len(query_list)):
        query_context_list.append(
            (
                query_list[i],
                doc_content_list[i]
            )
        )
    # get the answer
    results = client.batch_request_async_simple(
        query_context_list=query_context_list, 
        concurrency=5, 
        # model="Qwen/Qwen2.5-7B-Instruct", 
        model='Pro/Qwen/Qwen2.5-7B-Instruct',
        n = 1)
    extracted_answers = client.extract_answer(results)
    # extracted_answers format: list[list[str]]
    extracted_answers = [x[0] for x in extracted_answers]
    return extracted_answers

In [53]:
# generate answers
import nest_asyncio
import time
nest_asyncio.apply()

api_token = "sk-tiqxioukbterlpaezfbeezpfkcpqbtaqtknvfjrrkscgtwmg"
client = QwenChatClient(api_token=api_token)


def idx_answer(idx, valid_rank_data=valid_rank_data):
    all_answers = []
    # process 100 samples each time
    # 如果超出max_tokens，返回"None"
    batch_size = 50
    for i in range(0, len(valid_rank_data), batch_size):
        print(f"Processing {i} to {i + batch_size}")
        indices = get_answer(client, valid_rank_data[i:i + batch_size], idx=idx)
        all_answers.extend(indices)
        time.sleep(90)
    return all_answers


In [ ]:
all_answers0 = idx_answer(0)    

In [60]:
all_answers1 = idx_answer(1)

Processing 0 to 50
Extracting answers...


100%|██████████| 50/50 [00:00<00:00, 90006.52it/s]


Processing 50 to 100
Failed to request QwenChat API: {"code":20015,"message":"max_total_tokens (32906) must be less than or equal to max_seq_len (32768)","data":null}
we will return answer as "None"
Extracting answers...


100%|██████████| 50/50 [00:00<00:00, 144830.94it/s]


Processing 100 to 150
Extracting answers...


...

100%|██████████| 50/50 [00:00<00:00, 93248.20it/s]


Processing 900 to 950
Failed to request QwenChat API: {"code":20015,"message":"max_total_tokens (32996) must be less than or equal to max_seq_len (32768)","data":null}
we will return answer as "None"
Extracting answers...


100%|██████████| 50/50 [00:00<00:00, 122140.48it/s]


Processing 950 to 1000
Extracting answers...


100%|██████████| 50/50 [00:00<00:00, 153188.60it/s]


In [61]:
all_answers2 = idx_answer(2)

Processing 0 to 50
Extracting answers...


100%|██████████| 50/50 [00:00<00:00, 107878.19it/s]


Processing 50 to 100
Failed to request QwenChat API: {"code":20015,"message":"max_total_tokens (32996) must be less than or equal to max_seq_len (32768)","data":null}
we will return answer as "None"
Failed to request QwenChat API: {"code":20015,"message":"max_total_tokens (32905) must be less than or equal to max_seq_len (32768)","data":null}
we will return answer as "None"
Extracting answers...


100%|██████████| 50/50 [00:00<00:00, 134778.41it/s]


Processing 100 to 150
Extracting answers...


...

100%|██████████| 50/50 [00:00<00:00, 156737.82it/s]


Processing 900 to 950
Extracting answers...


100%|██████████| 50/50 [00:00<00:00, 135387.48it/s]


Processing 950 to 1000
Extracting answers...


100%|██████████| 50/50 [00:00<00:00, 108268.04it/s]


In [66]:
all_answers3 = idx_answer(3)

Processing 0 to 50
Extracting answers...


100%|██████████| 50/50 [00:00<00:00, 119632.17it/s]


Processing 50 to 100
Extracting answers...


100%|██████████| 50/50 [00:00<00:00, 104962.56it/s]


Processing 100 to 150
Extracting answers...


...

100%|██████████| 50/50 [00:00<00:00, 121292.77it/s]


Processing 900 to 950
Extracting answers...


100%|██████████| 50/50 [00:00<00:00, 58925.32it/s]


Processing 950 to 1000
Extracting answers...


100%|██████████| 50/50 [00:00<00:00, 82956.96it/s]


In [ ]:
all_answers4 = idx_answer(4)

In [ ]:
# # choose idx=0, idx=1, idx=2, idx=3, idx=4
# # we will get 5 answers, we check the answer in order
# # if answer == "None" or "" or "wrong" or "none", we will select the next answer
# # if all answers are not satisfied, we will select the first answer
# all_idx_answers = []
# for idx in range(5):
#     print(f"Processing {idx}")
#     all_answers = idx_answer(idx, valid_rank_data=valid_rank_data)
#     all_idx_answers.append(all_answers)

In [76]:
# check the answer
# all_idx_answers = ???
final_answers = []
all_idx_answers = [all_answers0, all_answers1, all_answers2, all_answers3]
for j in range(len(all_idx_answers[0])):
    tag = False
    for i in range(len(all_idx_answers)):
        if all_idx_answers[i][j] == "None" or all_idx_answers[i][j] == "" or all_idx_answers[i][j] == "wrong" or all_idx_answers[i][j] == "none":
            continue
        else:
            final_answers.append(all_idx_answers[i][j])
            tag = True
            break
    if tag is False:
        final_answers.append(all_idx_answers[0][j])


In [77]:
final_answers_np = np.array(final_answers)
final_answers_np.shape

(1000,)

In [78]:
# replace the attribute of 'answer' in valid_rank_data
valid_rank_data['answer'] = final_answers

In [79]:
# save valid_rank_data
valid_rank_data.to_json('data_and_code/zqy_data/faiss_results_rerank3.jsonl', orient='records', lines=True)

In [ ]:
# client = QwenChatClient(api_token=api_token)
# query_context_list = [("What is the best Chinese large model?", 
#                         "The Chinese large model industry is growing rapidly and has attracted a lot of attention. The industry is expected to face both opportunities and challenges in the coming years. The best model is Qwen."),
#                         ("What is the best Chinese large model?",
#                         "The Chinese large model industry is growing rapidly and has attracted a lot of attention. The industry is expected to face both opportunities and challenges in the coming years. The best model is deepseek."),
#                         ("What is the best Chinese large model?",
#                             "The Chinese large model industry is growing rapidly and has attracted a lot of attention. The industry is expected to face both opportunities and challenges in the coming years. The best model is hunyuan."),
#                         ]
# results = client.batch_request_async_simple(
#     query_context_list=query_context_list, 
#     concurrency=5, model="Qwen/Qwen2.5-7B-Instruct", 
#     n = 3)
# extracted_answers = client.extract_answer(results)